### Loading data and preliminary analysis

In [53]:
import pandas as pd
import nltk
import numpy as np
import re
from nltk.stem import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import pos_tag
from sklearn.metrics import pairwise_distances
from nltk import word_tokenize
from nltk.corpus import stopwords

In [54]:
from google.colab import files

# Upload the requirements.txt file
print("Please upload the 'requirements.txt' file.")
uploaded = files.upload()
import os
requirements_path = list(uploaded.keys())[0]
!pip install -r "{requirements_path}"
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


# Upload the Excel file
print("Please upload the Excel dataset file.")
uploaded = files.upload()

# Load the uploaded file into a DataFrame
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)


Please upload the 'requirements.txt' file.


Saving requirements.txt to requirements (2).txt
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached cachetools-5.3.2-py3-none-any.whl.metadata (5.2 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl.metadata (2.2 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached contourpy-1.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.9 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached fonttools-4.47.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (157 kB)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached grpcio-1.60.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
  Using cached h5py-3.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Saving mentalhealth.csv to mentalhealth (2).csv


In [55]:
df.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,"Mental illness does can affect anyone, regardl..."
2,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
3,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."
4,1619387,What should I do if I know someone who appears...,We encourage those with symptoms to talk to th...


In [56]:
df.isnull().sum()

,0
Question_ID,0
Questions,0
Answers,0


### Clean data using NLTK

In [57]:
# Sample string for tokenization
s = 'tell me about your personality'

# Tokenize the string
words = word_tokenize(s)
print(words)


['tell', 'me', 'about', 'your', 'personality']


In [58]:
# nltk.download('wordnet')
lemma = wordnet.WordNetLemmatizer()
lemma.lemmatize('absorbed', pos = 'v')

'absorb'

In [59]:
pos_tag(nltk.word_tokenize(s),tagset = None)

[('tell', 'VB'),
 ('me', 'PRP'),
 ('about', 'IN'),
 ('your', 'PRP$'),
 ('personality', 'NN')]

In [60]:
 nltk.download('stopwords')

stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
# function that performs text normalization steps and returns the lemmatized tokens as a sentence

def text_normalization(text):
    text = str(text).lower()
    spl_char_text = re.sub(r'[^ a-z]','',text)
    tokens = nltk.word_tokenize(spl_char_text)
    lema = wordnet.WordNetLemmatizer()
    tags_list = pos_tag(tokens,tagset=None)
    lema_words = []
    for token,pos_token in tags_list:
        if pos_token.startswith('V'):
            pos_val = 'v'
        elif pos_token.startswith('J'):
            pos_val = 'a'
        elif pos_token.startswith('R'):
            pos_val = 'r'
        else:
            pos_val = 'n'
        lema_token = lema.lemmatize(token,pos_val)

        if lema_token in stop:
          lema_words.append(lema_token)

    return " ".join(lema_words)

In [62]:
text_normalization('telling you some stuffs about me')

'you some about me'

In [63]:
df['lemmatized_text'] = df['Questions'].apply(text_normalization)
df.head(5)

,Question_ID,Questions,Answers,lemmatized_text
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...,what do it to have a
1,2110618,Who does mental illness affect?,"Mental illness does can affect anyone, regardl...",who do
2,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...,what be some of the of
3,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi...",can with
4,1619387,What should I do if I know someone who appears...,We encourage those with symptoms to talk to th...,what should i do if i who to have the of a


In [64]:
cv = CountVectorizer()                                  # intializing the count vectorizer
X = cv.fit_transform(df['lemmatized_text']).toarray()

In [65]:
# returns all the unique word from data

features = cv.get_feature_names_out()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head()

,about,after,an,and,any,be,before,between,but,can,...,to,too,we,what,when,where,who,why,with,you
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0


In [66]:
Question = 'What treatment options are available'                           # example

In [67]:
Question_lemma = text_normalization(Question)                               # clean text
Question_bow = cv.transform([Question_lemma]).toarray()                     # applying bow

### Cosine similarity

In [68]:
# cosine similarity for the above question we considered.

cosine_value = 1- pairwise_distances(df_bow, Question_bow, metric = 'cosine' )
(cosine_value)

array([[0.31622777],
       [0.        ],
       [0.5       ],
       [0.        ],
       [0.23570226],
       [0.        ],
       [1.        ],
       [0.31622777],
       [0.70710678],
       [0.        ],
       [0.31622777],
       [0.        ],
       [0.40824829],
       [0.25      ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.70710678],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [0.26726124],
       [0.        ],
       [0.40824829],
       [0.70710678],
       [0.25      ],
       [0.        ],
       [0.28867513],
       [0.23570226],
       [0.        ],
       [0.31622777],
       [0.        ],
       [0.35355339],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [69]:
df['similarity_bow'] = cosine_value                                         # create cosine value as a new column

In [70]:
simiscores = pd.DataFrame(df, columns=['Answers','similarity_bow'])         # taking similarity value of responses for the question we took
simiscores

,Answers,similarity_bow
0,Mental illnesses are health conditions that di...,0.316228
1,"Mental illness does can affect anyone, regardl...",0.000000
2,Symptoms of mental health disorders vary depen...,0.500000
3,"When healing from mental illness, early identi...",0.000000
4,We encourage those with symptoms to talk to th...,0.235702
...,...,...
92,Sorting out if you are drinking too much can b...,0.000000
93,"Cannabis smoke, for example, contains cancer-c...",0.500000
94,You can't. But you can influence their capacit...,0.000000
95,Cannabidiol or CBD is a naturally occurring co...,0.632456


In [71]:
simscoresDescending = simiscores.sort_values(by = 'similarity_bow', ascending=False)          # sorting the values
simscoresDescending.head()

,Answers,similarity_bow
78,Cyclothymic disorder is a subtype of bipolar d...,1.0
56,MSP stands for Medical Services Plan. It’s a h...,1.0
57,A referral means someone recommends you to ano...,1.0
88,Prodrome is a medical term for early signs or ...,1.0
6,Different treatment options are available for ...,1.0


In [72]:
threshold = 0.1                                                                         # considering the value of smiliarity to be greater than 0.1
df_threshold = simscoresDescending[simscoresDescending['similarity_bow'] > threshold]
df_threshold

,Answers,similarity_bow
78,Cyclothymic disorder is a subtype of bipolar d...,1.000000
56,MSP stands for Medical Services Plan. It’s a h...,1.000000
57,A referral means someone recommends you to ano...,1.000000
88,Prodrome is a medical term for early signs or ...,1.000000
6,Different treatment options are available for ...,1.000000
23,We all have mental health which is made up of ...,1.000000
84,Binge-eating disorder or BED is a type of eati...,1.000000
83,A personality disorder is a pattern of thought...,1.000000
82,A personality disorder is a pattern of thought...,1.000000
81,A personality disorder is a pattern of thought...,1.000000


In [73]:
index_value = cosine_value.argmax()         # index number of highest value
index_value

6

In [74]:
df['Answers'].loc[index_value]              # The text at the above index becomes the response for the question

'Different treatment options are available for individuals with mental illness.'

### Tf-Idf

In [75]:
Question1 = 'What treatment options are available'

In [76]:
# using tf-idf

tfidf = TfidfVectorizer()
x_tfidf = tfidf.fit_transform(df['lemmatized_text']).toarray()

In [77]:
Question_lemma1 = text_normalization(Question1)
Question_tfidf = tfidf.transform([Question_lemma1]).toarray()

In [78]:
# returns all the unique word from data with a score of that word

df_tfidf = pd.DataFrame(x_tfidf,columns = tfidf.get_feature_names_out())
df_tfidf.head()

,about,after,an,and,any,be,before,between,but,can,...,to,too,we,what,when,where,who,why,with,you
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.451394,0.0,0.0,0.302478,0.0,0.0,0.000000,0.0,0.00000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.877701,0.0,0.00000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.243612,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.213472,0.0,0.0,0.000000,0.0,0.00000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.407601,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.91316,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.309284,0.0,0.0,0.207251,0.0,0.0,0.471012,0.0,0.00000,0.0


In [79]:
cos = 1-pairwise_distances(df_tfidf,Question_tfidf,metric='cosine')                     # applying cosine similarity
cos

array([[0.19934817],
       [0.        ],
       [0.32390963],
       [0.        ],
       [0.13658848],
       [0.        ],
       [1.        ],
       [0.19419492],
       [0.60151332],
       [0.        ],
       [0.19419492],
       [0.        ],
       [0.20023572],
       [0.17842037],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.57482171],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [0.19775935],
       [0.        ],
       [0.29108153],
       [0.65904938],
       [0.17156074],
       [0.        ],
       [0.1796983 ],
       [0.14501087],
       [0.        ],
       [0.30481321],
       [0.        ],
       [0.28362797],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [80]:
df['similarity_tfidf'] = cos
df_simi_tfidf = pd.DataFrame(df, columns=['Answers','similarity_tfidf'])
df_simi_tfidf

,Answers,similarity_tfidf
0,Mental illnesses are health conditions that di...,0.199348
1,"Mental illness does can affect anyone, regardl...",0.000000
2,Symptoms of mental health disorders vary depen...,0.323910
3,"When healing from mental illness, early identi...",0.000000
4,We encourage those with symptoms to talk to th...,0.136588
...,...,...
92,Sorting out if you are drinking too much can b...,0.000000
93,"Cannabis smoke, for example, contains cancer-c...",0.370213
94,You can't. But you can influence their capacit...,0.000000
95,Cannabidiol or CBD is a naturally occurring co...,0.516687


In [81]:
df_simi_tfidf_sort = df_simi_tfidf.sort_values(by='similarity_tfidf', ascending=False)
df_simi_tfidf_sort.head(10)

,Answers,similarity_tfidf
78,Cyclothymic disorder is a subtype of bipolar d...,1.0
56,MSP stands for Medical Services Plan. It’s a h...,1.0
88,Prodrome is a medical term for early signs or ...,1.0
6,Different treatment options are available for ...,1.0
84,Binge-eating disorder or BED is a type of eati...,1.0
83,A personality disorder is a pattern of thought...,1.0
82,A personality disorder is a pattern of thought...,1.0
57,A referral means someone recommends you to ano...,1.0
81,A personality disorder is a pattern of thought...,1.0
23,We all have mental health which is made up of ...,1.0


In [82]:
threshold = 0.1                                                                                   # considering the value of smiliarity to be greater than 0.1
df_threshold = df_simi_tfidf_sort[df_simi_tfidf_sort['similarity_tfidf'] > threshold]
df_threshold

,Answers,similarity_tfidf
78,Cyclothymic disorder is a subtype of bipolar d...,1.000000
56,MSP stands for Medical Services Plan. It’s a h...,1.000000
88,Prodrome is a medical term for early signs or ...,1.000000
6,Different treatment options are available for ...,1.000000
84,Binge-eating disorder or BED is a type of eati...,1.000000
83,A personality disorder is a pattern of thought...,1.000000
82,A personality disorder is a pattern of thought...,1.000000
57,A referral means someone recommends you to ano...,1.000000
81,A personality disorder is a pattern of thought...,1.000000
23,We all have mental health which is made up of ...,1.000000


In [83]:
index_value1 = cos.argmax()                                                   # returns the index number of highest value
index_value1

6

In [84]:
df['Answers'].loc[index_value1]                                               # returns the text at that index

'Different treatment options are available for individuals with mental illness.'

### Testing chatbot

In [85]:
# defining a function that returns response to query using bow

def chat_bow(text):
    lemma = text_normalization(text)
    bow = cv.transform([lemma]).toarray()
    cosine_value = 1- pairwise_distances(df_bow,bow, metric = 'cosine' )
    index_value = cosine_value.argmax()
    return df['Answers'].loc[index_value]

In [86]:
chat_bow('can you prevent mental health problems')

'We can all suffer from mental health challenges, but developing our wellbeing, resilience, and seeking help early can help prevent challenges becoming serious.'

In [87]:
chat_bow('what is mental health')

'Different treatment options are available for individuals with mental illness.'

In [88]:
chat_bow('how do I know if i am unwell')

'If your beliefs , thoughts , feelings or behaviours have a significant impact on your ability to function in what might be considered a normal or ordinary way, it would be important to seek help.'

In [89]:
chat_bow('are there cures for mental health problems')

'It is often more realistic and helpful to find out what helps with the issues you face. Talking, counselling, medication, friendships, exercise, good sleep and nutrition, and meaningful occupation can all help.'

In [90]:
# defining a function that returns response to query using tf-idf

def chat_tfidf(text):
    lemma = text_normalization(text)
    tf = tfidf.transform([lemma]).toarray()
    cos = 1-pairwise_distances(df_tfidf,tf,metric='cosine')
    index_value = cos.argmax()
    return df['Answers'].loc[index_value]

In [91]:
chat_tfidf('how do i see a counsellor')

'If your beliefs , thoughts , feelings or behaviours have a significant impact on your ability to function in what might be considered a normal or ordinary way, it would be important to seek help.'

In [92]:
chat_tfidf('how to find a support group')

"Distraction is a very valid tool to help you cope when everything feels overwhelming or when you feel lonely or isolated. \n If you don't have a lot of energy or focus right now, try low-effort distractions like watching TV, browsing Youtube, listening to a podcast or audiobook, playing a game on your phone, reading an easy book or magazine, or working on a simple art project. \n If you have more energy and focus, give yourself a to-do list every day: you can clean and take care of projects around your home, work on hobbies, connect with family or friends, read a new book and catch up on your favourite TV shows. You can find interesting opportunities to take online courses from universities all over the world through MOOCs and other online learning platforms, you can learn a new language online or through apps, and you can learn new hobbies and activities. As more people have to practice social distancing or self-isolation, people are finding creative ways to bring the world into thei